In [16]:
!pip -qq install gradio

In [6]:
! pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 3.4 MB/s eta 0:00:00


In [7]:
import openai
import os
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
from tqdm import tqdm
import gradio

# Set your OpenAI API key here
os.environ["OPENAI_API_KEY"] = ""

#Extract Aspect,sentiment

In [8]:
def get_tag(sentence):
    client = openai.OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": """
            You are an aspect-based sentiment analysis tagger. Given a sentence, you need to return a JSON object of the following format:
            [
                {
                    "aspect_term": "<term>",
                    "aspect_category": "<category>",
                    "sentiment_term": "<term>",
                    "sentiment_category": "<category>"
                }
            ]
            If there are multiple aspect-sentiment pairs, include multiple elements in the array.

            Guidelines:
            - Aspect term should be some aspect for the product. It should not related to a concept or any human being
            - asperct term should be part of the senetences
            - Avoid redundant aspects. Each aspect should be unique.
            - The aspect term should clearly represent a distinct part or feature of the sentence.
            - The sentiment term should reflect the sentiment related to the aspect.
            - Ensure that categories are concise and relevant.
            - output should be a valid json and no other keywords

            Example:
            Input: "The food was great but the service was terrible."
            Output: [
                {
                    "aspect_term": "food",
                    "aspect_category": "food_quality",
                    "sentiment_term": "great",
                    "sentiment_category": "positive"
                },
                {
                    "aspect_term": "service",
                    "aspect_category": "service_quality",
                    "sentiment_term": "terrible",
                    "sentiment_category": "negative"
                }
            ],
            Input: "The Pizza was great but the room was terrible."
            Output: [
                {
                    "aspect_term": "food",
                    "aspect_category": "food_quality",
                    "sentiment_term": "great",
                    "sentiment_category": "positive"
                },
                {
                    "aspect_term": "ambience",
                    "aspect_category": "ambience",
                    "sentiment_term": "terrible",
                    "sentiment_category": "negative"
                }
            ]
            """},
            {"role": "user", "content": sentence}
        ]
    )
    #return response.choices[0].text.strip()
    return response.choices[0].message.content


# Test on one example

In [11]:

def extract_aspects(review,row_id=1):
    json_output = get_tag(review)
    aspects_data = json.loads(json_output)
    aspects_df= pd.DataFrame(aspects_data)
    aspects_df['review']=review
    aspects_df['row_id']=row_id
    return aspects_df


In [86]:
extract_aspects('i love food')

,aspect_term,aspect_category,sentiment_term,sentiment_category,review,row_id
0,food,food_quality,love,positive,i love food,1


In [12]:
comment=('''I had the white KindleDX, and I have to confess, I mainly wanted this one because it was black . . . more manly, y\'know, but it gave me an excuse to pass on the white one to a friend who might not otherwise have bought one.  I can\'t even tell when it\'s using 3G, or if that\'s just for overseas, but it seems to work flawlessly, and I think everything (good) they say about it is true.  Downloads books in less than a minute, and if you have a periodical subscription, it\'s like a present every time a new one arrives, and you open your Kindle and find it there.<br /><br />I especially like the "collections" feature, which is kinda like putting folders on your computer, because you can make up your own categories and then plop the books in there, in as many collections as you like.  Thus, you could have, say, a "psychology & spirituality" collection, and a "philosophy" collection, and if one of your books seems to fit both categories,
 you can add it to both, and then turn around and
 create another collection by a specific author and add it to that,
 too!<br /><br />With the extensive selection in the Kindle bookstore,
 you really have a resource just for looking up information as well,
 ' whether you\'re buying or not.  And I\'ve used the limited web
  browser in a pinch as well.<br /><br />I LOVE my Black KindleDX!!!''')

In [14]:
extract_aspects(comment)

,aspect_term,aspect_category,sentiment_term,sentiment_category,review,row_id
0,white KindleDX,product_color,white,neutral,"I had the white KindleDX, and I have to confes...",1
1,black KindleDX,product_color,LOVE,positive,"I had the white KindleDX, and I have to confes...",1
2,3G,connectivity,flawlessly,positive,"I had the white KindleDX, and I have to confes...",1
3,downloads books,performance,less than a minute,positive,"I had the white KindleDX, and I have to confes...",1
4,periodical subscription,features,present,positive,"I had the white KindleDX, and I have to confes...",1
5,collections feature,features,especially like,positive,"I had the white KindleDX, and I have to confes...",1
6,Kindle bookstore,selection,extensive,positive,"I had the white KindleDX, and I have to confes...",1
7,limited web browser,features,used in a pinch,positive,"I had the white KindleDX, and I have to confes...",1


#Read review data

In [19]:
df=pd.read_csv('sample_data/electronics_reviews.csv')

In [21]:
df.columns

Index(['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id',
       'timestamp', 'helpful_vote', 'verified_purchase'],
      dtype='object')

In [22]:
df['row_id'] = range(1, len(df) + 1)

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   rating             5000 non-null   float64
 1   title              4999 non-null   object 
 2   text               5000 non-null   object 
 3   images             5000 non-null   object 
 4   asin               5000 non-null   object 
 5   parent_asin        5000 non-null   object 
 6   user_id            5000 non-null   object 
 7   timestamp          5000 non-null   int64  
 8   helpful_vote       5000 non-null   int64  
 9   verified_purchase  5000 non-null   bool   
 10  row_id             5000 non-null   int64  
dtypes: bool(1), float64(1), int64(3), object(6)
memory usage: 395.6+ KB


In [27]:
df['text'] = df['text'].astype(str)

In [87]:
df.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,row_id
0,5.0,Buen vendedor,Todo perfecto,[],B06XWZWYVP,B071R715MZ,AGTUTBAETJADNOTZNQMDM6MD7EUA,1550165327367,0,True,1
1,4.0,Easy to set up,Sound is good quality. I like it that although...,"[{'attachment_type': 'IMAGE', 'large_image_url...",B094HGQKFY,B096N43C98,AFM6YFPTEMBQ5EJYNX5QIFV6ZTPQ,1644514482866,2,True,2
2,5.0,Five Stars,DOES EXACTLY WHAT IT IS INTENDED FOR.,[],B00CIOA89E,B00CIOA89E,AFVHI5KJMQW42B4OLQ7O4MQCLJYA,1407881213000,0,True,3
3,5.0,Clear sharp picture,The TV antennae is quick to install and driver...,[],B07P6YY1KK,B07P6YY1KK,AHMXFTTVTPM7BU7MUBGLRAHHHZTA,1602586461351,1,True,4
4,4.0,"Great value, Great Product","Super easy to setup, took less then a minute. ...",[],B005A0B7CA,B005A0B7CA,AEK5CGQAJAK5TL4EBO5NNOVZHXLA,1405331888000,0,True,5


#Data analysis

check how many unique parent asin we have

filter out top 100 asin

In [23]:
asin_counts = df['parent_asin'].value_counts()

In [24]:
top_100_asins = asin_counts.head(100).index.tolist()

In [25]:
top_100_asin_df = df[df['parent_asin'].isin(top_100_asins)]

In [26]:
top_100_asin_df['parent_asin'].value_counts().sum()

415

In [47]:
top_100_data=top_100_asin_df[0:100]

In [48]:
top_100_data['parent_asin'].nunique()

58

# Extract aspect sentiment for 100 data  (unique parent asin 58 )

In [28]:
def process_data(df):
    arr = []
    for i,x in tqdm(df.iterrows()):
        try:
            review = x['text']
            row_id = x['row_id']
            result = get_tag(review)
            tags = json.loads(result)
            for t in tags:
                t['row_id'] = row_id
                arr.append(t)
        except:
            print(i)

    return pd.DataFrame(arr)

In [31]:
aspect_data=process_data(top_100_asin_df[0:100])

8it [00:19,  1.84s/it]

76


10it [00:20,  1.19s/it]

80


11it [00:20,  1.03it/s]

...

38it [01:04,  1.73s/it]

410


46it [01:16,  1.52s/it]

485


100it [03:04,  1.84s/it]


In [32]:
aspect_data.head()

,aspect_term,aspect_category,sentiment_term,sentiment_category,row_id
0,Amazon 4K Fire Stick,product,LOVE,positive,6
1,TV,device,out of date,negative,6
2,fire stick,usability,brief pause in streaming,negative,6
3,wifi connection,connectivity,due to,neutral,6
4,fire stick,usability,like it so much,positive,6


# Merging the aspect sentiment information with original electronics data

In [33]:
final_data=pd.merge(top_100_asin_df,aspect_data,on='row_id',how='inner')

In [34]:
final_data.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,row_id,aspect_term,aspect_category,sentiment_term,sentiment_category
0,5.0,A MUST HAVE if your TV is over 3 years old!!!!!,I LOVE my Amazon 4K Fire Stick!!! My TV is on...,[],B079QHML21,B07GZFM1ZM,AH2ZSLHQVBDWUNOSNGNL7A7MBZQA,1607909896367,0,True,6,Amazon 4K Fire Stick,product,LOVE,positive
1,5.0,A MUST HAVE if your TV is over 3 years old!!!!!,I LOVE my Amazon 4K Fire Stick!!! My TV is on...,[],B079QHML21,B07GZFM1ZM,AH2ZSLHQVBDWUNOSNGNL7A7MBZQA,1607909896367,0,True,6,TV,device,out of date,negative
2,5.0,A MUST HAVE if your TV is over 3 years old!!!!!,I LOVE my Amazon 4K Fire Stick!!! My TV is on...,[],B079QHML21,B07GZFM1ZM,AH2ZSLHQVBDWUNOSNGNL7A7MBZQA,1607909896367,0,True,6,fire stick,usability,brief pause in streaming,negative
3,5.0,A MUST HAVE if your TV is over 3 years old!!!!!,I LOVE my Amazon 4K Fire Stick!!! My TV is on...,[],B079QHML21,B07GZFM1ZM,AH2ZSLHQVBDWUNOSNGNL7A7MBZQA,1607909896367,0,True,6,wifi connection,connectivity,due to,neutral
4,5.0,A MUST HAVE if your TV is over 3 years old!!!!!,I LOVE my Amazon 4K Fire Stick!!! My TV is on...,[],B079QHML21,B07GZFM1ZM,AH2ZSLHQVBDWUNOSNGNL7A7MBZQA,1607909896367,0,True,6,fire stick,usability,like it so much,positive


In [77]:
top_100_asin_df[top_100_asin_df['parent_asin']=='B002GYWHSQ']['text']

213     I had the white KindleDX, and I have to confes...
886     I heard that the Kindle was the best e-book re...
2308    I love the convenience and reliability of Amaz...
Name: text, dtype: object

In [78]:
top_100_asin_df['text'][213]

'I had the white KindleDX, and I have to confess, I mainly wanted this one because it was black . . . more manly, y\'know, but it gave me an excuse to pass on the white one to a friend who might not otherwise have bought one.  I can\'t even tell when it\'s using 3G, or if that\'s just for overseas, but it seems to work flawlessly, and I think everything (good) they say about it is true.  Downloads books in less than a minute, and if you have a periodical subscription, it\'s like a present every time a new one arrives, and you open your Kindle and find it there.<br /><br />I especially like the "collections" feature, which is kinda like putting folders on your computer, because you can make up your own categories and then plop the books in there, in as many collections as you like.  Thus, you could have, say, a "psychology & spirituality" collection, and a "philosophy" collection, and if one of your books seems to fit both categories, you can add it to both, and then turn around and cre

In [49]:
#final_data[final_data['parent_asin']=='B002GYWHSQ']

#final_data.to_csv("sample_data/final_aspect_data.csv")

#Gradio for demonstartion

In [38]:
def get_html_output(aspects_data):
    html_output = """
    <style>
        table {
            width: 100%;
            border-collapse: collapse;
            margin: 20px 0;
            font-size: 18px;
            text-align: left;
        }
        table th, table td {
            padding: 12px;
            border: 1px solid #ddd;
        }
        table th {
            background-color: #f2f2f2;
        }
        table tr:nth-child(even) {
            background-color: #f9f9f9;
        }
        table tr:hover {
            background-color: #f1f1f1;
        }
        .positive {
            background-color: #d4edda;
        }
        .negative {
            background-color: #f8d7da;
        }
    </style>
    <table>
        <tr>
            <th>Aspect</th>
            <th>Aspect Category</th>
            <th>Sentiment</th>
            <th>Sentiment Category</th>
        </tr>"""

    for item in aspects_data:
        sentiment_class = "positive" if item['sentiment_category'] == "positive" else "negative" if item['sentiment_category'] == "negative" else ""
        html_output += f"<tr><td>{item['aspect_term']}</td><td>{item['aspect_category']}</td><td class='{sentiment_class}'>{item['sentiment_term']}</td><td class='{sentiment_class}'>{item['sentiment_category']}</td></tr>"

    html_output += "</table>"

    return html_output


In [42]:
def aspect_extraction_gradio(review):
    json_output = get_tag(review)
    aspects_data = json.loads(json_output)
    return get_html_output(aspects_data)


In [ ]:
in_review = gradio.Textbox(lines=2, placeholder=None, value="review", label='Enter Review Text')

In [ ]:
#out_label = gradio.Textbox(type="text", label='Aspect_Label')

In [44]:
iface = gradio.Interface(fn = aspect_extraction_gradio,
                         inputs = in_review,
                         outputs = gradio.HTML())

iface.launch(share = True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b389e86d0ce3743f2b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
